### index preservation

In [1]:
import pandas as pd
import numpy as np

rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [2]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)), columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


In [4]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [3]:
1+np.log(df/10)

,A,B,C,D
0,0.489174,0.894639,-0.609438,0.489174
1,0.643325,0.083709,-0.203973,0.643325
2,0.643325,-0.609438,0.306853,0.083709


### index alignment

In [30]:
gdp = pd.Series([5974.7, 10031.0, 14681.1, 25116.3], 
                   index=pd.date_range('2023', periods=4, freq='QE'))
inflation = pd.Series([0.015, 0.025, 0.025, 0.025],
                     index=pd.date_range('2023-06-30', periods=4, freq='QE'))

In [31]:
gdp

2023-03-31     5974.7
2023-06-30    10031.0
2023-09-30    14681.1
2023-12-31    25116.3
Freq: QE-DEC, dtype: float64

In [32]:
inflation

2023-06-30    0.015
2023-09-30    0.025
2023-12-31    0.025
2024-03-31    0.025
Freq: QE-DEC, dtype: float64

In [58]:
pd.concat([gdp, inflation], axis=1)

,0,1
2023-03-31,5974.7,NaN
2023-06-30,10031.0,0.015
2023-09-30,14681.1,0.025
2023-12-31,25116.3,0.025
2024-03-31,NaN,0.025


In [33]:
gdp/(1+inflation)

2023-03-31             NaN
2023-06-30     9882.758621
2023-09-30    14323.024390
2023-12-31    24503.707317
2024-03-31             NaN
Freq: QE-DEC, dtype: float64

In [59]:
# fills in missing values with 1 for missing values on both sides
gdp.div(1+inflation, fill_value= 1) 

2023-03-31     5974.700000
2023-06-30     9882.758621
2023-09-30    14323.024390
2023-12-31    24503.707317
2024-03-31        0.975610
Freq: QE-DEC, dtype: float64

In [61]:
1/(1+0.025)

0.9756097560975611

In [34]:
set(gdp.index) | set(inflation.index)

{Timestamp('2023-03-31 00:00:00'),
 Timestamp('2023-06-30 00:00:00'),
 Timestamp('2023-09-30 00:00:00'),
 Timestamp('2023-12-31 00:00:00'),
 Timestamp('2024-03-31 00:00:00')}

### index alignment in dataframe

In [43]:
econ_df = pd.DataFrame({'gdp': gdp, 'inflation': inflation})
pop_df = pd.DataFrame({'population': [35.467, 63.951, 80.94, 328.2]},
                        index=pd.date_range('2022-09-30', periods=4, freq='QE'))

In [44]:
pop_df

,population
2022-09-30,35.467
2022-12-31,63.951
2023-03-31,80.940
2023-06-30,328.200


In [51]:
comb = pd.concat([econ_df, pop_df], axis=1).sort_index()
comb

,gdp,inflation,population
2022-09-30,NaN,NaN,35.467
2022-12-31,NaN,NaN,63.951
2023-03-31,5974.7,NaN,80.940
2023-06-30,10031.0,0.015,328.200
2023-09-30,14681.1,0.025,NaN
2023-12-31,25116.3,0.025,NaN
2024-03-31,NaN,0.025,NaN


In [54]:
comb / econ_df

,gdp,inflation,population
2022-09-30,NaN,NaN,NaN
2022-12-31,NaN,NaN,NaN
2023-03-31,1.0,NaN,NaN
2023-06-30,1.0,1.0,NaN
2023-09-30,1.0,1.0,NaN
2023-12-31,1.0,1.0,NaN
2024-03-31,NaN,1.0,NaN


In [65]:
comb.div(econ_df,fill_value=1)

,gdp,inflation,population
2022-09-30,NaN,NaN,35.467
2022-12-31,NaN,NaN,63.951
2023-03-31,1.0,NaN,80.940
2023-06-30,1.0,1.0,328.200
2023-09-30,1.0,1.0,NaN
2023-12-31,1.0,1.0,NaN
2024-03-31,NaN,1.0,NaN


### ops b/w series and dataframes

In [66]:
A = rng.randint(10, size=(3, 4))
A

array([[1, 7, 5, 1],
       [4, 0, 9, 5],
       [8, 0, 9, 2]])

In [67]:
A - A[0]

array([[ 0,  0,  0,  0],
       [ 3, -7,  4,  4],
       [ 7, -7,  4,  1]])

In [73]:
A - A[:,1][:, np.newaxis]

array([[-6,  0, -2, -6],
       [ 4,  0,  9,  5],
       [ 8,  0,  9,  2]])

In [68]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,3,-7,4,4
2,7,-7,4,1


In [69]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-6,0,-2,-6
1,4,0,9,5
2,8,0,9,2


In [74]:
#automatically aligns the data
halfrow = df.iloc[0, ::2]
halfrow

Q    1
S    5
Name: 0, dtype: int32

In [75]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,3.0,NaN,4.0,NaN
2,7.0,NaN,4.0,NaN
